Eléonor KIOULOU
Khadija MOKHTARI

#PROJET NLP

In [ ]:
!pip install transformers

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Télécharge l'ensemble des fichiers et du code du référentiel sur votre environnement local
!git clone https://github.com/cr-nlp/project1-2023.git

fatal: destination path 'project1-2023' already exists and is not an empty directory.


In [ ]:
# Chargement des fichiers
dir = "./project1-2023/"
queries_file_path = dir + "dev.all.queries"
docs_file_path = dir + "dev.docs"

## BERT

In [ ]:
# Importation des bibliothèques nécessaires
import urllib.request as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
import nltk

# Téléchargement des ressources nécessaires pour nltk
nltk.download('stopwords')
nltk.download('punkt')

# Fonction pour charger le corpus NFCorpus
def loadNFCorpus():
    # Répertoire du projet
    dir = "./project1-2023/"

    # Chargement des documents
    filename = dir + "dev.docs"
    dicDoc = {}
    with open(filename) as file:
        lines = file.readlines()
    for line in lines:
        tabLine = line.split('\t')
        key = tabLine[0]
        value = tabLine[1]
        dicDoc[key] = value

    # Chargement des requêtes
    filename = dir + "dev.all.queries"
    dicReq = {}
    with open(filename) as file:
        lines = file.readlines()
    for line in lines:
        tabLine = line.split('\t')
        key = tabLine[0]
        value = tabLine[1]
        dicReq[key] = value

    # Chargement des scores de pertinence entre requêtes et documents
    filename = dir + "dev.2-1-0.qrel"
    dicReqDoc = defaultdict(dict)
    with open(filename) as file:
        lines = file.readlines()
    for line in lines:
        tabLine = line.strip().split('\t')
        req = tabLine[0]
        doc = tabLine[2]
        score = int(tabLine[3])
        dicReqDoc[req][doc] = score

    # Retourner les dictionnaires chargés
    return dicDoc, dicReq, dicReqDoc


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Importation des bibliothèques nécessaires
from transformers import BertTokenizer, BertModel
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from collections import defaultdict

# Fonction pour exécuter le modèle BERT
def run_bert_only(startDoc, endDoc):
    # Chargement du corpus NFCorpus
    dicDoc, dicReq, dicReqDoc = loadNFCorpus()

    # Initialisation du tokenizer et du modèle DistilBERT
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertModel.from_pretrained('distilbert-base-uncased')

    # Initialisation des listes et dictionnaires nécessaires
    docsToKeep = []
    reqsToKeep = []
    dicReqDocToKeep = defaultdict(dict)

    # Itération sur les requêtes et documents pour sélectionner une plage spécifique
    i = startDoc
    for reqId in dicReqDoc:
        if i > (endDoc - startDoc):
            break
        for docId in dicReqDoc[reqId]:
            dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
            docsToKeep.append(docId)
            i = i + 1
        reqsToKeep.append(reqId)
    docsToKeep = list(set(docsToKeep))

    # Calcul des embeddings pour les documents
    corpusDocEmbeddings = {}
    for k in docsToKeep:
        docTokens = tokenizer(dicDoc[k], return_tensors='pt', padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            output = model(**docTokens)
        embeddings = output.last_hidden_state.mean(dim=1).squeeze().numpy()
        corpusDocEmbeddings[k] = embeddings

    # Calcul des embeddings pour les requêtes
    corpusReqEmbeddings = {}
    for k in reqsToKeep:
        reqTokens = tokenizer(dicReq[k], return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            output = model(**reqTokens)
        embeddings = output.last_hidden_state.mean(dim=1).squeeze().numpy()
        corpusReqEmbeddings[k] = embeddings

    # Retour des embeddings pour les documents et les requêtes
    return corpusDocEmbeddings, corpusReqEmbeddings

# Exécution de la fonction avec une plage spécifique de documents
nb_docs = 3192  # tous les documents
corpusDocEmbeddings, corpusReqEmbeddings = run_bert_only(0, nb_docs)


In [ ]:
# Parcourir toutes les clés (identifiants de document) dans corpusDocEmbeddings
PLAIN_list = []  # Liste pour stocker les identifiants de requête
MED_list = []    # Liste pour stocker les identifiants de document

# Itération sur les embeddings des documents
for doc_id, embedding in corpusDocEmbeddings.items():
    MED_list.append(doc_id)

# Itération sur les embeddings des requêtes
for req_id, embedding in corpusReqEmbeddings.items():
    PLAIN_list.append(req_id)

# Conversion des listes en ensembles pour éliminer les doublons, puis en listes à nouveau
PLAIN_list = list(set(PLAIN_list))
MED_list = list(set(MED_list))

# Affichage des identifiants de requête et de document
print("MED: \n", MED_list, "\nPLAIN:\n", PLAIN_list)


MED: 
 ['MED-3485', 'MED-4589', 'MED-2106', 'MED-2891', 'MED-2053', 'MED-4095', 'MED-3770', 'MED-3635', 'MED-4071', 'MED-2424', 'MED-1812', 'MED-2704', 'MED-4038', 'MED-1846', 'MED-2170', 'MED-951', 'MED-1216', 'MED-2144', 'MED-3541', 'MED-2772', 'MED-3874', 'MED-2521', 'MED-2985', 'MED-4124', 'MED-2276', 'MED-4349', 'MED-5070', 'MED-3745', 'MED-874', 'MED-1188', 'MED-5171', 'MED-2340', 'MED-3591', 'MED-2215', 'MED-3669', 'MED-5016', 'MED-2356', 'MED-5059', 'MED-2657', 'MED-2208', 'MED-3376', 'MED-4186', 'MED-4174', 'MED-3318', 'MED-1832', 'MED-2925', 'MED-3373', 'MED-2077', 'MED-4159', 'MED-2479', 'MED-2367', 'MED-2344', 'MED-2446', 'MED-1267', 'MED-3820', 'MED-4144', 'MED-2154', 'MED-3475', 'MED-2089', 'MED-1843', 'MED-1740', 'MED-3757', 'MED-4158', 'MED-2219', 'MED-5081', 'MED-4512', 'MED-4276', 'MED-4556', 'MED-1718', 'MED-3840', 'MED-3708', 'MED-2021', 'MED-4523', 'MED-4649', 'MED-4412', 'MED-2035', 'MED-4222', 'MED-4600', 'MED-3728', 'MED-3453', 'MED-1416', 'MED-2138', 'MED-1289'

In [ ]:
filename = dir + "dev.2-1-0.qrel"
dicReqDoc = defaultdict(dict)

# Charger le fichier dev.2-1-0.qrel et construire le dictionnaire des scores de pertinence
with open(filename) as file:
    lines = file.readlines()

relevance = []  # Liste pour stocker les scores de pertinence
for line in lines:
    tabLine = line.strip().split('\t')

    req = tabLine[0]
    doc = tabLine[2]
    score = int(tabLine[3])
    dicReqDoc[req][doc] = score

# Parcourir toutes les requêtes dans dicReqDoc
for req, scores_pertinence in dicReqDoc.items():

    # Ajouter les scores de pertinence à la liste
    relevance.extend(list(scores_pertinence.values()))

# Afficher la liste complète des scores de pertinence ainsi que les listes d'identifiants de requête et de document
print("Scores de pertinence : ", relevance)
print("Identifiants de requête : ", PLAIN_list)
print("Identifiants de document : ", MED_list)


Scores de pertinence :  [2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(query_embedding, doc_embedding):
    # Calculer la similarité cosinus entre les embeddings de la requête et du document
    return cosine_similarity([query_embedding], [doc_embedding])[0][0]

# Exemple d'utilisation :
doc_id = "MED-4798"
req_id = "PLAIN-55"

# Récupérer les embeddings du document et de la requête
doc_embedding = corpusDocEmbeddings[doc_id]
req_embedding = corpusReqEmbeddings[req_id]

# Calculer la similarité entre la requête et le document
similarity = calculate_similarity(req_embedding, doc_embedding)

# Afficher la similarité
print(f"Similarité entre la requête '{req_id}' et le document '{doc_id}': {similarity}")


Similarité entre la requête 'PLAIN-55' et le document 'MED-4798': 0.8105282783508301


## NDCG@5

In [ ]:
similarity_list = []

# Parcourir toutes les combinaisons de requêtes et de documents
for plain in PLAIN_list:
    for med in MED_list:
        # Récupérer les embeddings du document et de la requête
        doc_id = med
        req_id = plain
        doc_embedding = corpusDocEmbeddings[doc_id]
        req_embedding = corpusReqEmbeddings[req_id]

        # Calculer la similarité entre la requête et le document
        similarity = calculate_similarity(req_embedding, doc_embedding)

        # Ajouter la similarité et l'identifiant du document à la liste
        similarity_list.append([similarity, doc_id])

# Afficher la liste complète des similarités
print(similarity_list)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Triez la liste de manière décroissante
sorted_list = sorted(similarity_list, key=lambda x: x[0], reverse=True)

# Prenez les 5 meilleurs scores
top_5_scores = sorted_list[:5]
best_scores = []
combinaison = []

for s in top_5_scores:
  best_scores.append(s[0])
  combinaison.append(s[1])

# Affichez les 5 meilleurs scores
print("Les 5 meilleurs scores sont :", top_5_scores)
print(best_scores)
print(combinaison)

Les 5 meilleurs scores sont : [[0.9040048, 'MED-4057'], [0.9039674, 'MED-2075'], [0.9025656, 'MED-4777'], [0.90246713, 'MED-1254'], [0.90001506, 'MED-3681']]
[0.9040048, 0.9039674, 0.9025656, 0.90246713, 0.90001506]
['MED-4057', 'MED-2075', 'MED-4777', 'MED-1254', 'MED-3681']


In [ ]:
# Liste des termes MED
med_only = combinaison

# Dictionnaire pour stocker les relevances
relevances_med = {}

# Chemin vers le fichier qrel
filename_qrel = dir + "dev.2-1-0.qrel"

# Lire le fichier qrel et extraire les relevances
with open(filename_qrel) as file:
    lines = file.readlines()
    for line in lines:
        tabLine = line.strip().split('\t')
        req = tabLine[0]
        doc = tabLine[2]
        score = int(tabLine[3])
        # Vérifier si le document commence par "MED" et est dans la liste des termes MED
        for element in combinaison:
          if (doc == element):
            # Stocker la relevance dans le dictionnaire
            relevances_med[doc] = score

# Afficher les relevances
print("Relevances des termes MED:", relevances_med)


Relevances des termes MED: {'MED-3681': 1, 'MED-4777': 1, 'MED-2075': 1, 'MED-1254': 1, 'MED-4057': 1}


In [ ]:
from math import log2

def ndcg_at_k(predictions, relevance, k):
    # Trier les prédictions et les scores de pertinence par ordre décroissant
    sorted_preds = [p for _, p in sorted(zip(relevance, predictions), reverse=True)]
    sorted_relevance = sorted(relevance, reverse=True)

    # Calculer DCG@k (Discounted Cumulative Gain at k)
    dcg_at_k = sum([(2**p - 1) / (log2(i + 2)) for i, p in enumerate(sorted_preds[:k])])

    # Calculer IDCG@k (Ideal Discounted Cumulative Gain at k)
    idcg_at_k = sum([(2**p - 1) / (log2(i + 2)) for i, p in enumerate(sorted_relevance[:k])])

    # Calculer NDCG@k (Normalized Discounted Cumulative Gain at k)
    ndcg_at_k = dcg_at_k / idcg_at_k if idcg_at_k > 0 else 0

    return ndcg_at_k

predictions = best_scores
relevance = list(relevances_med.values())
ndcg_5 = ndcg_at_k(predictions, relevance, k=5)
print("NDCG@5:", ndcg_5)


NDCG@5: 0.869957234097763
